미세조정 전후 비교

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["긍정", "부정"]

# 미세 조정전 모델(랜덤 초기화)
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_before = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME)
model_before.eval()  #평가모드
with torch.no_grad():
    for i, review in enumerate(test_reviews):
        inputs = tokenizer(review,return_tensors='pt')
        outputs =  model_before(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits,dim=-1)[0]
        pred_label = torch.argmax(probs).item()

        print(f'리뷰 : {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]}')
        print(f'긍정 : {probs[0]} | 부정 : {probs[1]}\n')

c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


리뷰 : 1 : This movie is absolutely amazing! Best film I've ever seen.
예측 : 부정 0.6160548329353333
긍정 : 0.38394519686698914 | 부정 : 0.6160548329353333

리뷰 : 2 : Terrible waste of time. Worst movie ever made.
예측 : 부정 0.6263812184333801
긍정 : 0.37361881136894226 | 부정 : 0.6263812184333801

리뷰 : 3 : It was okay, nothing special but not bad either.
예측 : 부정 0.5642200112342834
긍정 : 0.4357799291610718 | 부정 : 0.5642200112342834

리뷰 : 4 : Brilliant acting and stunning visuals throughout!
예측 : 부정 0.6433290839195251
긍정 : 0.35667091608047485 | 부정 : 0.6433290839195251

리뷰 : 5 : Boring and predictable plot. Very disappointed.
예측 : 부정 0.600482702255249
긍정 : 0.3995172679424286 | 부정 : 0.600482702255249



In [2]:
# 학습 - 미세조정
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
train_texts = [
    ("This is wonderful and fantastic!", 1),  # 긍정
    ("Absolutely terrible and awful!", 0),     # 부정
    ("I love this so much!", 1),
    ("Hated it completely!", 0)
]
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_after = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME,num_labels=2)
from transformers import AdamW
optimizer = AdamW(model_after.parameters(), lr=2e-5)
model_after.train()
for epoch in range(7):
    total_loss = 0
    for text,label in train_texts:
        optimizer.zero_grad()
        inputs = tokenizer(text,return_tensors='pt')
        labels = torch.tensor([label])
        outputs = model_after(**inputs,labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(train_texts)
    print(f'epoch : {epoch+1}  loss : {avg_loss:.4f}')    

c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable th

epoch : 1  loss : 0.8576
epoch : 2  loss : 0.5143
epoch : 3  loss : 0.4029
epoch : 4  loss : 0.3566
epoch : 5  loss : 0.3656
epoch : 6  loss : 0.2884
epoch : 7  loss : 0.2703


In [3]:
print(f'미세조정 후')
test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["긍정", "부정"]

with torch.no_grad():
    for i, review in enumerate(test_reviews):
        inputs = tokenizer(review,return_tensors='pt')
        outputs =  model_after(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits,dim=-1)[0]
        pred_label = torch.argmax(probs).item()        
        print(f'리뷰 : {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]}')
        print(f'긍정 : {probs[0]} | 부정 : {probs[1]}\n')

미세조정 후
리뷰 : 1 : This movie is absolutely amazing! Best film I've ever seen.
예측 : 부정 0.7634645700454712
긍정 : 0.2365354597568512 | 부정 : 0.7634645700454712

리뷰 : 2 : Terrible waste of time. Worst movie ever made.
예측 : 긍정 0.8025669455528259
긍정 : 0.8025669455528259 | 부정 : 0.1974330097436905

리뷰 : 3 : It was okay, nothing special but not bad either.
예측 : 부정 0.7303644418716431
긍정 : 0.2696356177330017 | 부정 : 0.7303644418716431

리뷰 : 4 : Brilliant acting and stunning visuals throughout!
예측 : 긍정 0.6486348509788513
긍정 : 0.6486348509788513 | 부정 : 0.35136517882347107

리뷰 : 5 : Boring and predictable plot. Very disappointed.
예측 : 긍정 0.7610774636268616
긍정 : 0.7610774636268616 | 부정 : 0.23892255127429962

